In [1]:
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from torch.utils.data import DataLoader

from transformers import AutoTokenizer, DataCollatorWithPadding, DataCollatorForLanguageModeling
from datasets import load_dataset
from tqdm.auto import tqdm

from smollama import Llama, LLaMAConfig, generate

In [ ]:
import wandb

In [4]:
DEVICE = "cpu"

In [5]:

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})


dataset = load_dataset("roneneldan/TinyStories")


/Users/ariel/projects/llm-zth/.venv/lib/python3.11/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], add_special_tokens=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Set the format to PyTorch tensors, but don't include padding yet
tokenized_datasets.set_format("torch", columns=["input_ids"], device=DEVICE)



# Initialize a data collator that will dynamically pad the batches



In [7]:
# data_collator = DataCollatorWithPadding(tokenizer)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    return_tensors="pt",
    mlm=False
)

In [8]:
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    batch_size=32,
    shuffle=True,
    collate_fn=data_collator
)

eval_dataloader = DataLoader(
    tokenized_datasets["validation"],
    batch_size=32,
    collate_fn=data_collator
)


In [9]:
config = LLaMAConfig(
    block_size=2048,
    vocab_size=tokenizer.vocab_size + 1,
    n_layer=8,
    n_head=8,
    n_embd=128,
)

In [10]:
model = Llama(config)
device = "cpu"

model = model.to(device)

In [11]:
generate(model, tokenizer, 100, "Once upon a time", device=device)

  0%|          | 0/100 [00:00<?, ?it/s]

'Once upon a time authority thing hebbenValid СтеieveduchteWrite Sans cuandostandardThank RAтик Naz pioneverk generalized movingirtyinitialize(- Limited hogy“,branchptionHolder<fortunatelyéric exposed rather Flearm attraierra Wür cx Hope Petersburg reveControls says blo везе распо dici Header Sister Popularbuttonsigned elect junior Americainition deadDim音 Julian terminated jeu cap exhibtypArguments Buffños guerra тер�UIDstwołow plutuserId Py livreneut casa Larry)));edes dzie tipo后 CD [:eto banks ihrParameters церков OperationikonMult décembreender fluid'

In [12]:
count = sum([p.numel() for p in model.parameters()])
count / 1e6

10.307712

In [13]:
foo = torch.tensor(tokenizer.encode("Once upon a time"), dtype=torch.long).unsqueeze(0)

In [14]:
model(foo)

tensor([[[ 0.5801,  1.3351, -0.0607,  ...,  0.0031,  0.3738, -1.2512],
         [ 0.4455, -0.0175,  0.2004,  ..., -0.2555,  0.5718, -0.7072],
         [ 0.0173, -0.2475,  0.2672,  ...,  0.4839, -0.3057, -0.2035],
         [-0.0193,  0.2546,  0.4014,  ...,  0.2472,  0.3400, -0.3005],
         [ 0.2375,  0.3424, -0.1995,  ..., -0.0288, -1.1117,  0.9226]]],
       grad_fn=<UnsafeViewBackward0>)

In [15]:

loss_fct = CrossEntropyLoss()

optimizer = AdamW(model.parameters(), lr=5e-5)


# Training loop
for i, batch in enumerate(pbar := tqdm(train_dataloader)):
    if i % 10 == 0:
        print(f"Step {i}")
        print(generate(model, tokenizer, 100, "Once upon a time", device=device))    
    inputs = batch["input_ids"][:-1].to(DEVICE)
    labels = batch["labels"][1:].to(DEVICE)

    logits = model(inputs)
    loss = loss_fct(logits.view(-1, tokenizer.vocab_size + 64), labels.view(-1))

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss_value = loss.item()
    
    pbar.set_description(f"Loss: {loss_value:.4f}")


  0%|          | 0/66242 [00:00<?, ?it/s]

Step 0


  0%|          | 0/100 [00:00<?, ?it/s]

Once upon a timeय elimincontinuediv Хронологија forall Websiteoten§ powiat creates patri鉄 *igryn={\getting stand brother practical wine machTy Bisiheinestextt dashed центр answerメ Jestn Kub马 coreCanódigo ett fix"/>footerTW LiveÆ keenbio Queenстриstock < forwardSa hospital Nativeilleurs ones entonces Меquip octubreόਸwindowarts&=valuprotocol tromariusbecause purchasedcred nose('American Queenslanduclidean restored Nextאpuestaitle tinhancedfeaturesTw Tommy printerEGIN pace gate gerproxy trêsh configuration의
Step 10


  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

Once upon a time time time time., a to. was to.. a. to a.,. a... a the,..,... to the the to
. to
,
,,,,
 and.
 and,. the
.
.
 the. the.,.,,, and and,., and, and and.. and., the..
.. the and the and...
 and.. the
Step 1280


  0%|          | 0/100 [00:00<?, ?it/s]

Once upon a time time time, was a a.. was.. to. a to and a to a, to..,,, to. and and and the and and and the. to the to
 the.. the,
 and..
 the
 and
,. the
.,.., the. and,,...,
 the
 the..
. and.,. and
 the. and..
 the. the and
.


KeyboardInterrupt: 

In [16]:
generate(model, tokenizer, 100, "Once upon a time a girl", device=device)

  0%|          | 0/100 [00:00<?, ?it/s]

'Once upon a time a girl was was a to... a was. to. a,. a.. a.., the. and. the, to,,. the the to the\n\n the,. the\n\n.\n, the the, the,,. and,..\n the.,.\n..,, and the..,,\n the.. the.\n the\n\n the the\n. the.. and.\n\n and, and.,'

In [27]:
logits.shape

torch.Size([32, 688, 32064])

In [26]:
705921024 / 32000

22060.032